In [10]:
import pandas as pd
import re

# Read the list of filenames from the configuration file
with open('file_list.txt', 'r', encoding='utf-8') as config_file:
    file_names = config_file.read().splitlines()

# Regex pattern to match the data format
pattern = r'\[(.*?)\] (.*?): (.*)'


# Initialize an empty list to store parsed data
datalist = []
stream_count = 0
# Iterate over each specified file
for file in file_names:
    full_path = file
    with open(full_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            match = re.match(pattern, line)
            if match:
                date, user, message = match.groups()
                datalist.append([date, user, message,stream_count])
    stream_count = stream_count + 1

# Create a DataFrame from the parsed data
data = pd.DataFrame(datalist, columns=["date", "user", "message","stream"])

data['date'] = pd.to_datetime(data['date'])

data = data.sort_values(by=['user', 'date'])

data2 = data[data['user'] == 'balintboss']


# Calculate the total number of messages per stream
total_messages_per_stream = data2['stream'].value_counts()

# Combine the results into a single DataFrame
stream_stats = pd.DataFrame({
    'total_messages': total_messages_per_stream
})

# Calculate the time difference between the first and last messages for each stream
watch_lengths = data2.groupby('stream')['date'].agg(lambda x: (x.max() - x.min()).total_seconds())

# Add the stream_length column to the stream_stats DataFrame
stream_stats['watch_length'] = watch_lengths

# Calculate the time difference between the first and last messages for each stream
stream_lengths = data.groupby('stream')['date'].agg(lambda x: (x.max() - x.min()).total_seconds())

# Add the stream_length column to the stream_stats DataFrame
stream_stats['stream_length'] = stream_lengths

stream_stats.head(10)

,total_messages,watch_length,stream_length
stream,,,
25,105,35147.0,36105.0
49,87,22521.0,22548.0
60,86,18002.0,18233.0
20,79,17236.0,17472.0
19,79,20342.0,21336.0
5,76,20013.0,20382.0
18,69,6995.0,7430.0
48,67,25516.0,25912.0
42,65,21074.0,21262.0


In [11]:
stream_stats.describe()

,total_messages,watch_length,stream_length
count,62.000000,62.000000,62.000000
mean,41.483871,17784.193548,18305.758065
std,21.603538,5541.879538,5540.839191
min,5.000000,2861.000000,5257.000000
25%,25.250000,15703.250000,16421.750000
50%,40.500000,17642.500000,18032.000000
75%,54.500000,20747.250000,21205.500000
max,105.000000,35147.000000,36105.000000
